In [ ]:
from google.colab import drive
drive._mount('/content/drive')

In [ ]:
!pip install sentence-transformers

In [ ]:
!pip install fasttext

#**Importing packages**

In [ ]:
import pandas as pd
import numpy as np
import string
import fasttext
import fasttext.util
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn import model_selection, naive_bayes
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.linear_model import LogisticRegression 
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sentence_transformers import SentenceTransformer
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.metrics import classification_report,confusion_matrix

#**English dataset**

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/Datasets/homo-trans/splited data/mini project II/eng/eng_train.csv')
test = pd.read_csv('/content/drive/MyDrive/Datasets/homo-trans/splited data/mini project II/eng/eng_test.csv')

In [ ]:
train.value_counts('category')

In [ ]:
test.value_counts('category')

#**Removing punctuations**

In [ ]:
def remove_punctuations(txt):
    text_nopunc="".join([c for c in txt if c not in string.punctuation])
    return text_nopunc

train['text']=train['text'].apply(lambda x: remove_punctuations(x))
test['text']=test['text'].apply(lambda x: remove_punctuations(x))

#**Encoding**

In [ ]:
Encoder = LabelEncoder()
train['category']=Encoder.fit_transform(train['category'])
test['category']=Encoder.fit_transform(test['category'])

#**fastText**

In [ ]:
fasttext.util.download_model('en', if_exists='ignore')  # english
T_ft = fasttext.load_model('cc.en.300.bin')

T_ft.get_dimension()
fasttext.util.reduce_model(T_ft, 100)

In [ ]:
f_train = [T_ft.get_word_vector(words)for words in X_train]
f_test = [T_ft.get_word_vector(words)for words in X_test]

#**Naive Bayes**

In [ ]:
naive_bayes = BernoulliNB()
# naive_bayes = MultinomialNB()
naive_bayes.fit(f_train, y_train)
NB_pred = naive_bayes.predict(f_test)

In [ ]:
print(confusion_matrix(y_test, NB_pred)) 
print(classification_report(y_test,NB_pred))

#**Support Vector Machine**

In [ ]:
# normal SVM
SVM = SVC(C=10, kernel = 'linear', degree = 3, class_weight='balanced'   , gamma = 'auto')
X_Train_svm = SVM.fit(f_train, y_train)

y_pred_svm = SVM.predict(f_test)

In [ ]:
print(confusion_matrix(y_test,y_pred_svm)) 
print(classification_report(y_test, y_pred_svm))

#**Logistic Regression**

In [ ]:
# fit the training dataset on the LR classifier
lr = LogisticRegression(C=2,class_weight = 'balanced',max_iter=1000)
lr.fit(f_train,y_train)

# predict the dev dataset
pred_LR = lr.predict(f_test)

In [ ]:
print(confusion_matrix(y_test,pred_LR)) 
print(classification_report(y_test,pred_LR))

#**Random Forest**

In [ ]:
sc_X = StandardScaler(with_mean=False)
X_Train_RF = sc_X.fit_transform(f_train)
test_RF = sc_X.transform(f_test)
classifier = RandomForestClassifier(n_estimators=10, class_weight='balanced')
classifier.fit(X_Train_RF, y_train)
Y_Pred = classifier.predict(test_RF)

In [ ]:
print(confusion_matrix(y_test,Y_Pred)) 
print(classification_report(y_test,Y_Pred))

#**Decision Tree**

In [ ]:
clf = DecisionTreeClassifier(criterion='entropy', max_depth=5, random_state=0)
clf = clf.fit(f_train, y_train)
y_Pred = clf.predict(f_test)

In [ ]:
print(confusion_matrix(y_test,y_Pred)) 
print(classification_report(y_test,y_Pred))

#**XGBoost**

In [ ]:
xgboost = XGBClassifier()
xgboost.fit(np.array(f_train), y_train)
pred_xg = xgboost.predict(f_test)

In [ ]:
print(confusion_matrix(y_test, pred_xg)) 
print(classification_report(y_test,pred_xg))